In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
!pip install tensorflow
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras import backend as K

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 9.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 18.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 20.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.4 MB/s eta 0:00:00a 0:00:01


In [3]:
train_X = pd.read_csv('/Users/Haley/Desktop/WiDs Datathon/WiDs Notebooks/X_test.csv')
test_X = pd.read_csv('WiDs Notebooks/X_train.csv')
Y_train =  pd.read_excel('/Users/Haley/Desktop/WiDs Datathon/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')


In [9]:
train_X = train_X.drop(columns = ['participant_id'])
test_X = test_X.drop(columns = ['participant_id'])
Y_train = Y_train.drop(columns = ['participant_id'])

In [10]:
# Assume you already loaded:
# train_X, Y_train, test_X

# Scale features (important for neural nets)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_X)
X_test_scaled = scaler.transform(test_X)

# Define Neural Network
input_dim = X_train_scaled.shape[1]

model = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='sigmoid')  # Two outputs: ADHD_Outcome, Sex_F
])



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Basic_Demos_Enroll_Year_2016
- Basic_Demos_Enroll_Year_2017
- Basic_Demos_Enroll_Year_2018
- Basic_Demos_Enroll_Year_2019
- Basic_Demos_Study_Site_2
- ...
Feature names seen at fit time, yet now missing:
- Basic_Demos_Enroll_Year_2021
- Basic_Demos_Enroll_Year_2022
- Basic_Demos_Enroll_Year_2023
- Basic_Demos_Study_Site_5
- MRI_Track_Scan_Location_4


In [ ]:


# Custom F1 Score
def f1_score_metric(y_true, y_pred):
    y_pred_binary = K.round(y_pred)  # round predictions to 0 or 1
    tp = K.sum(K.cast(y_true * y_pred_binary, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_binary, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_binary), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)  # average across ADHD and Sex




In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[f1_score_metric]  # instead of just accuracy
)


# Train on all training data
history = model.fit(
    X_train_scaled,
    Y_train,
    epochs=50,
    batch_size=32,
    callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)]
)

# Predict on test set
y_test_probs = model.predict(X_test_scaled)
y_test_pred = (y_test_probs > 0.5).astype(int)

# y_test_pred now has two columns: first column ADHD_Outcome, second column Sex_F

# You can prepare your submission file
submission = pd.DataFrame({
    'ADHD_Outcome': y_test_pred[:,0],
    'Sex_F': y_test_pred[:,1]
})

# Save submission
submission.to_csv('submission.csv', index=False)

keras.callbacks.ModelCheckpoint(
    'best_model.h5', 
    monitor='f1_score_metric', 
    mode='max', 
    save_best_only=True, 
    save_weights_only=True
)
